# Speed up chance calculation for score matrix

author: steeve.laquitaine@epfl.ch  
last modified: 13-02-2024

**Method**:

* **delta_time (Δ𝑡)** = 1.3 ms: the time windows before and after the spike timestamp of a ground truth. When a the timestamp of a sorted unit falls within this time window, they coincide and the sorted timestamp is a hit.
* **chance level score**: see paper
* **dark (missed) units**: ground truth units with sorting accuracy below the chance agreement score (their best match with a sorted unit produce an agreement score below chance).
* **false positive units**: units which timestamps never hit the timestamps of the ground truth units wihin 50 microns of the probe: they never fall withing the delta_time window.


### Setup

Create or activate env `spikeinterf...`

In [25]:
%load_ext autoreload
%autoreload 2
import os 
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd
import seaborn as sns 
import spikeinterface as si
from spikeinterface import comparison
import copy
from concurrent.futures import ProcessPoolExecutor
proj_path = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(proj_path)

from src.nodes.postpro.cell_matching import get_SpikeInterface_matching_object
from src.nodes.utils import get_config
from src.nodes.postpro.feateng import (add_firing_rates)
from src.nodes.analysis.failures import accuracy as acc
from src.nodes.metrics.metrics import get_firing_rate

# PARAMETERS
REC_DURATION = 600 # 10 minutes recording
DET = 0.8
CHANCE_THRESH = 0.1

# DATASETS

# NPX
# Synthetic
# cfg_nb, _ = get_config("buccino_2020", "2020").values()
# GT_nb_10m = cfg_nb["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
# KS3_nb = cfg_nb["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
# REC_nb = cfg_nb["probe_wiring"]["full"]["output"]

# biophy spont
cfg_ns, _ = get_config("silico_neuropixels", "concatenated").values()
KS3_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
GT_ns_10m = cfg_ns["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
REC_ns = cfg_ns["probe_wiring"]["full"]["output"]

# biophy evoked
# cfg_ne, _ = get_config("silico_neuropixels", "stimulus").values()
# KS4_ne = cfg_ne["sorting"]["sorters"]["kilosort3"]["output"]
# GT_ne_10m = cfg_ne["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
# REC_ne = cfg_ne["probe_wiring"]["full"]["output"]

# FIGURE SETTINGS
COLOR_VIVO = (0.7, 0.7, 0.7)
COLOR_SILI = (0.84, 0.27, 0.2)
COLOR_STIM = (0.6, 0.75, 0.1)
BOX_ASPECT = 1                  # square fig
FIG_SIZE = (1,1)
plt.rcParams['figure.figsize'] = (2,1)
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6
plt.rcParams['lines.linewidth'] = 0.2
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['xtick.major.width'] = 0.3
plt.rcParams['xtick.minor.size'] = 0.1
plt.rcParams['xtick.major.size'] = 1.5
plt.rcParams['ytick.major.size'] = 1.5
plt.rcParams['ytick.major.width'] = 0.3
legend_cfg = {"frameon": False, "handletextpad": 0.1}
savefig_cfg = {"transparent":True}
# print(plt.rcParams.keys())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-08-26 18:17:27,792 - root - utils.py - get_config - INFO - Reading experiment config.
2024-08-26 18:17:27,963 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Chance score

In [26]:
# calculate chance score for a 1 Hz sorted unit (spikes/secs)
fr_gt = 1
fr_s = 1
delta_time = 1.3  # in ms
rec_dur = 600  # recording duration

# calculate chance agreement score
# - chance probability of hits
# - chance score
p_chance_hit = acc.get_p_chance_hit(1 / 1000, 1.3)
chance_acc = acc.get_unit_chance_agreement_score(fr_gt, fr_s, 600, p_chance_hit)

### Define custom functions

In [27]:
def classify_sorted_unit_biases(agreem_mx, det):
    """_summary_

    Args:
        agreem_mx (_type_): _description_

    Returns:
        (dict):
        - "match" (pd.DataFrame): N ground truth unit indices, N best-match sorted units

    """

    # note: with this approach (BEST matching approach), the same sorted unit can be paired with more than one true unit
    # we only keep the pairings with highest agreement scores
    # true-sorted unit pairing
    pairing = agreem_mx.T.idxmax(axis=1)
    pairing = pairing.to_frame()
    pairing.columns = ["sorted"]

    # add agreement score
    accuracy = agreem_mx.T.max(axis=1)
    pairing["accuracy"] = accuracy

    # check if the only sorted unit paired with this true unit
    sorted_ids = agreem_mx.index

    df = copy.copy(pairing.iloc[0, :].to_frame().T)
    false_positives = []

    # else keep the pairing with highest agreement score
    # loop over all sorted single unit units
    for ix in range(len(sorted_ids)):
        # case the sorted unit was paired with a ground truth unit
        if any(pairing["sorted"] == sorted_ids[ix]):
            sorted_pairings = pairing[pairing["sorted"] == sorted_ids[ix]].sort_values(
                by="accuracy", ascending=False
            )
            # take max pairing (first row)
            df = pd.concat([df, sorted_pairings.iloc[0, :].to_frame().T])
        else:
            # case the sorted unit was paired with none of the ground truth units
            false_positives.append(sorted_ids[ix])

    df = df[1:]
    df["sorted"] = df["sorted"].astype(int)

    # count biases
    n_good = sum(df["accuracy"] >= det)
    n_poor = sum((df["accuracy"] >= CHANCE_THRESH) & (df["accuracy"] < det))
    n_below_chance = sum((df["accuracy"] > 0) & (df["accuracy"] < CHANCE_THRESH))
    n_false_pos = len(false_positives)

    # sanity check
    # assert n_good + n_poor + n_below_chance + n_false_pos == len(
    #     sorted_ids
    # ), "They must match"
    return {
        "n_good": n_good,
        "n_poor": n_poor,
        "n_below_chance": n_below_chance,
        "n_false_pos": n_false_pos,
        "match": pairing,
    }

    """_summary_

    Some scores can be 0, if they were the best found

    Args:
        scores (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    # the same sorted unit can be paired with more than one true unit
    # we keep the pairings with highest agreement scores
    # true-sorted unit pairing
    match = scores.idxmax(axis=0)
    match = match.to_frame()
    match.columns = ["sorted"]
    match.index.name = "true"

    # record agreement score
    match["accuracy"] = scores.max(axis=0)
    return match


def count_match(match: pd.DataFrame, sorted: int):
    """count the number of true units
    one sorted unit matches

    Args:
        match (pd.DataFrame):
        - index: true units id
        - columns:
        - - "sorted": best-match sorted unit id
        - - "accuracy": best-match agreement score
        (true unit sorting accuracy)
    """
    return sum(match["sorted"] == sorted)


def get_best_matched_true_unit(match, s_id):
    """get true unit matchs

    Args:
        match (_type_): _description_
        s_id (_type_): _description_

    Returns:
        _type_: _description_
    """
    units = match[match["sorted"] == s_id].index.values
    if len(units) == 1:
        units = units[0]
    return units


def get_all_matched_true_unit(s_id, scores):
    """get all true unit matchs

    Args:
        match (_type_): _description_
        s_id (_type_): _description_

    Returns:
        _type_: _description_
    """
    return np.array(scores.columns[scores.loc[s_id, :] > 0])


def get_chance_score(gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time):
    fr_s = get_firing_rate(s_id, Sorting_ns, duration)
    fr_gt = get_firing_rate(gt_id, SortingTrue_ns, duration)
    p_chance_hit = acc.get_p_chance_hit(min(fr_gt, fr_s) / 1000, delta_time)
    return acc.get_unit_chance_agreement_score(fr_gt, fr_s, duration, p_chance_hit)


def get_score(scores, s_id, gt_id):
    try:
        return scores.loc[s_id, gt_id]
    except:
        from ipdb import set_trace

        set_trace()


def is_score_at_chance(
    scores, s_id, gt_id, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    score = get_score(scores, s_id, gt_id)
    chance = get_chance_score(
        gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
    )
    return score <= chance


def are_all_scores_at_chance(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check whether a sorted unit agreement scores with the many
    ground truth units it matches are all at chance (it is a false
    positive)

    Args:
        s_id (_type_): sorted unit id
        gt_id (_type_): ground truth unit id
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """

    # make an 1D array
    gt_id = make_1darray(gt_id)

    # start counting scores below chance
    d = 0
    for _, g_i in enumerate(gt_id):
        d += is_score_at_chance(
            scores, s_id, g_i, Sorting_ns, SortingTrue_ns, duration, delta_time
        )
    return d == len(gt_id)


def make_1darray(gt_id):
    if isinstance(gt_id, np.ndarray):
        return gt_id
    else:
        return np.array([gt_id])


def is_oversplitter_1(
    scores, match, s_id, gt_id, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check when sorted unit matches only one ground truth unit

    Args:
        scores (_type_): _description_
        match (_type_): _description_
        s_id (_type_): _description_
        gt_id (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    # find all sorted units that match this ground truth unit
    sorteds = match[match.index == gt_id]["sorted"].values.tolist()

    d = 0
    # loop over sorted unit match
    for _, s_i in enumerate(sorteds):

        # check for other sorted unit match
        # than the target match s_id
        if not s_i == s_id:
            d += is_score_at_chance(
                scores,
                s_i,
                gt_id,
                Sorting_ns,
                SortingTrue_ns,
                duration,
                delta_time,
            )
    return d > 0


def is_oversplitter_2(
    s_id, gt_id, scores, match, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check when sorted unit matches many ground truth units

    Args:
        s_id (_type_): _description_
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    d = 0
    for _, g_i in enumerate(gt_id):
        d += is_oversplitter_1(
            scores,
            match,
            s_id,
            g_i,
            Sorting_ns,
            SortingTrue_ns,
            duration,
            delta_time,
        )
    return d > 0


def is_poor(det, scores, gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time):
    """is poor (score between chance and threshold for "good" (80%))

    Args:
        scores (_type_): _description_
        gt_id (_type_): _description_
        s_id (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    chance = get_chance_score(
        gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
    )
    score = get_score(scores, s_id, gt_id)
    return (score > chance) & (score < det)


def is_poor_2(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time, det
):
    """is poor when sorted unit matches many ground truth units
    (all its scores with the ground truth are below the "good" threshold
    and it has at least one score with a ground truth above chance

    Args:
        s_id (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_
        det (_type_): _description_

    Returns:
        _type_: _description_
    """
    chance_all = []
    score_all = []

    for _, g_i in enumerate(gt_id):
        chance_all.append(
            get_chance_score(
                g_i, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
            )
        )
        score_all.append(get_score(scores, s_id, g_i))

    # all scores are below DET
    test_1 = all(np.array(score_all) < det)

    # at least one score is above chance
    test_2 = any(np.array(score_all) > np.array(chance_all))
    return test_1 & test_2


def is_overmerger_2(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """is an overmerger: the sorted unit matches at least two ground
    truths with above chance scores

    Args:
        s_id (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    # get there scores
    chance_all = []
    score_all = []

    for _, g_i in enumerate(gt_id):
        # get chance scores
        chance_all.append(
            get_chance_score(
                g_i, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
            )
        )
        # get scores
        score_all.append(get_score(scores, s_id, g_i))
    return sum(np.array(score_all) > np.array(chance_all)) > 1


def is_good_2(s_id, gt_id, scores):
    """is sorted unit good: it matches one ground truth with 80% and no
    other ground truth with

    Args:
        s_id (_type_): _description_
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    score_all = []
    for _, g_i in enumerate(gt_id):
        score_all.append(get_score(scores, s_id, g_i))
    return any(np.array(score_all) >= DET)


def set_df(df, sorted, true, quality, score):
    df.loc[sorted, "sorted"] = sorted
    df.loc[sorted, "true"] = true
    df.loc[sorted, "score"] = score
    qual = df.loc[sorted, "quality"]
    # record quality if nan (empty)
    # else append new quality
    if isinstance(qual, str):
        df.loc[sorted, "quality"] += quality
    else:
        df.loc[sorted, "quality"] = quality
    return df


def get_scores(
    SortingTrue,
    Sorting,
    delta_time: float,
):
    comp = comparison.compare_sorter_to_ground_truth(
        SortingTrue,
        Sorting,
        exhaustive_gt=True,
        delta_time=delta_time,
    )
    # return comp.agreement_scores.max(axis=1).sort_values(ascending=False).values
    return comp.agreement_scores

## False positives

In [35]:
# load SortingExtractors
SortingTrue_ns = si.load_extractor(GT_ns_10m)
Sorting_ns = si.load_extractor(KS3_ns_10m)
Rec_ns = si.load_extractor(REC_ns)

# get scores (N sorted units rows x N true units columns)
scores_ns = get_scores(SortingTrue_ns, Sorting_ns, 1.3)
scores_ns = scores_ns.T

# curate (get single-unit only)
scores_ns = scores_ns.loc[
    Sorting_ns.unit_ids[Sorting_ns.get_property("KSLabel") == "good"], :
]

scores_by_exp = [scores_ns]

In [21]:
# compute sorting quality
out = classify_sorted_unit_biases(scores_ns, DET)

**Method**:

* For each true unit we find its best-match sorted unit
* calculate chance level for all units is pairs is consuming-
* all NaN scores are sorted units' will many ground truth matches.
* 15 minutes


In [6]:
scores = scores_ns
scores

,12165,16652,18371,19690,21040,24768,29248,30168,32331,37423,...,4213917,4215563,4216128,4217493,4221920,4223302,4225319,4228700,4229218,4229506
143,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.001927
201,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001315,0.000804,0.000649,0.0,0.001650,0.0,0.0,0.001612,0.0,0.000878
208,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001019,0.002314,0.000000,0.0,0.000391,0.0,0.0,0.000000,0.0,0.000000
215,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001202,0.000451,0.001184,0.0,0.000970,0.0,0.0,0.000391,0.0,0.000000
217,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000622,0.000282,0.000000,0.0,0.000453,0.0,0.0,0.000599,0.0,0.002375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1612,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000964,0.001316,0.001433,0.0,0.000842,0.0,0.0,0.001188,0.0,0.000590
1613,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.002236,0.000537,0.000000,0.0,0.001676,0.0,0.0,0.000000,0.0,0.000000
1632,0.000109,0.0,0.000109,0.0,0.0,0.0,0.0,0.000109,0.0,0.0,...,0.001555,0.002293,0.001065,0.0,0.001194,0.0,0.0,0.001063,0.0,0.000525


In [29]:
from scipy.stats import poisson
from joblib import Parallel, delayed
from tqdm import tqdm

# import warnings

# warnings.filterwarnings("ignore", category=DeprecationWarning)


# def get_p_chance_hit(fr: float, delta_time: float):
#     """derive the chance probability of hits
#     (coincidences between two independent sorted and
#     ground truth unit spike trains)

#     We should use the firing rate of the less firing
#     of the two. It determines the expected maximum
#     possible number of coincidences.

#     Args:
#         fr (float): firing rate in spikes/ms
#         delta_time (float): SpikeInterface delta_time interval in ms

#     Returns:
#         _type_: _description_
#     """
#     k = 0  # we want the probability of k=0 coincidences
#     interval_ms = 2 * delta_time  # time interval in ms
#     n_sp = interval_ms * fr  # expected nb of coincidences
#     return 1.0 - poisson.pmf(k=k, mu=n_sp)


def get_unit_chance_agreement_score(fr_gt: float, fr_s: float, rec_dur: float):
    """get unit chance scorey

    The chance scorey metrics should change with the ground truth firing rate.
    It is not the case with the current calculation.
    Intuition: the more a ground truth unit spikes within the duration of recording (say 600 secs),
    the more spikes will be missed when compared a sorting unit of a fixed firing rate.
    The increasing number of misses should decrease the value of the chance score metrics,
    which is currently not the case.

    Args:
        fr_gt (float): ground truth firing rate (spikes/secs)
        fr_s (float): sorted unit firing rate (spikes/secs)
        p_chance_hit (float): chance probability of hits
        rec_dur (float): recording duration
    """

    k = 0  # we want the probability of k=0 coincidences
    interval_ms = 2 * delta_time  # time interval in ms
    n_sp = interval_ms * min(fr_gt, fr_s) / 1000  # expected nb of coincidences
    p_chance_hit = 1.0 - poisson.pmf(k=k, mu=n_sp)

    # nb of spikes
    n_gt = fr_gt * rec_dur
    n_s = fr_s * rec_dur

    # nb of hits, false positives, misses
    # - the smallers spike train min(n_gt, n_s) determines
    # the maximum possible number of hits
    n_h = p_chance_hit * min(n_gt, n_s)
    n_fp = n_s - n_h
    n_m = n_gt - n_h
    return n_h / (n_h + n_m + n_fp)


def get_chance_score(gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time):

    # firing rates
    fr_s = Sorting_ns.count_num_spikes_per_unit()[s_id] / duration
    fr_gt = SortingTrue_ns.count_num_spikes_per_unit()[gt_id] / duration

    # calculate p_chance_hit
    k = 0  # we want the probability of k=0 coincidences
    interval_ms = 2 * delta_time  # time interval in ms
    n_sp = interval_ms * min(fr_gt, fr_s) / 1000  # expected nb of coincidences
    p_chance_hit = 1.0 - poisson.pmf(k=k, mu=n_sp)

    # nb of spikes
    n_gt = fr_gt * duration
    n_s = fr_s * duration

    # nb of hits, false positives, misses
    # - the smallers spike train min(n_gt, n_s) determines
    # the maximum possible number of hits
    n_h = p_chance_hit * min(n_gt, n_s)
    n_fp = n_s - n_h
    n_m = n_gt - n_h
    return n_h / (n_h + n_m + n_fp)


def get_firing_rate(unit_id: int, Sorting, rec_duration: float):
    """get a unit's firing rate

    Returns:
        (float): firing rate in spikes/secs
    """
    n_spikes = Sorting.count_num_spikes_per_unit()[unit_id]
    return n_spikes / rec_duration


# # warning: all get_chance_score's underlying functions must be in the same module
# chance_df = Parallel(n_jobs=-1)(
#     delayed(get_chance_score)(i, j, Sorting_ns, SortingTrue_ns, 600, delta_time)
#     for _, i in enumerate(tqdm(scores.columns))
#     for j in scores.index
# )

In [32]:
get_chance_score(4213917, 143, Sorting_ns, SortingTrue_ns, 600, delta_time)

2.7021588890723976e-06

In [17]:
def classify_sorted_unit_biases(agreem_mx, det):
    """_summary_

    Args:
        agreem_mx (_type_): _description_

    Returns:
        (dict):
        - "match" (pd.DataFrame): N ground truth unit indices, N best-match sorted units

    """

    # note: with this approach (BEST matching approach), the same sorted unit can be paired with more than one true unit
    # we only keep the pairings with highest agreement scores
    # true-sorted unit pairing
    pairing = agreem_mx.T.idxmax(axis=1)
    pairing = pairing.to_frame()
    pairing.columns = ["sorted"]

    # add agreement score
    accuracy = agreem_mx.T.max(axis=1)
    pairing["accuracy"] = accuracy

    # check if the only sorted unit paired with this true unit
    sorted_ids = agreem_mx.index

    df = copy.copy(pairing.iloc[0, :].to_frame().T)
    false_positives = []

    # else keep the pairing with highest agreement score
    # loop over all sorted single unit units
    for ix in range(len(sorted_ids)):
        # case the sorted unit was paired with a ground truth unit
        if any(pairing["sorted"] == sorted_ids[ix]):
            sorted_pairings = pairing[pairing["sorted"] == sorted_ids[ix]].sort_values(
                by="accuracy", ascending=False
            )
            # take max pairing (first row)
            df = pd.concat([df, sorted_pairings.iloc[0, :].to_frame().T])
        else:
            # case the sorted unit was paired with none of the ground truth units
            false_positives.append(sorted_ids[ix])

    df = df[1:]
    df["sorted"] = df["sorted"].astype(int)

    # count biases
    n_good = sum(df["accuracy"] >= det)
    n_poor = sum((df["accuracy"] >= CHANCE_THRESH) & (df["accuracy"] < det))
    n_below_chance = sum((df["accuracy"] > 0) & (df["accuracy"] < CHANCE_THRESH))
    n_false_pos = len(false_positives)

    # sanity check
    # assert n_good + n_poor + n_below_chance + n_false_pos == len(
    #     sorted_ids
    # ), "They must match"
    return {
        "n_good": n_good,
        "n_poor": n_poor,
        "n_below_chance": n_below_chance,
        "n_false_pos": n_false_pos,
        "match": pairing,
    }


def count_match(match: pd.DataFrame, sorted: int):
    """count the number of true units
    one sorted unit matches

    Args:
        match (pd.DataFrame):
        - index: true units id
        - columns:
        - - "sorted": best-match sorted unit id
        - - "accuracy": best-match agreement score
        (true unit sorting accuracy)
    """
    return sum(match["sorted"] == sorted)


def get_best_matched_true_unit(match, s_id):
    """get true unit matchs

    Args:
        match (_type_): _description_
        s_id (_type_): _description_

    Returns:
        _type_: _description_
    """
    units = match[match["sorted"] == s_id].index.values
    if len(units) == 1:
        units = units[0]
    return units


def get_all_matched_true_unit(s_id, scores):
    """get all true unit matchs

    Args:
        match (_type_): _description_
        s_id (_type_): _description_

    Returns:
        _type_: _description_
    """
    return np.array(scores.columns[scores.loc[s_id, :] > 0])


def get_chance_score(gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time):
    fr_s = get_firing_rate(s_id, Sorting_ns, duration)
    fr_gt = get_firing_rate(gt_id, SortingTrue_ns, duration)
    p_chance_hit = acc.get_p_chance_hit(min(fr_gt, fr_s) / 1000, delta_time)
    return acc.get_unit_chance_agreement_score(fr_gt, fr_s, duration, p_chance_hit)


def get_score(scores, s_id, gt_id):
    try:
        return scores.loc[s_id, gt_id]
    except:
        from ipdb import set_trace

        set_trace()


def is_score_at_chance(
    scores, s_id, gt_id, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    score = get_score(scores, s_id, gt_id)
    chance = get_chance_score(
        gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
    )
    return score <= chance


def are_all_scores_at_chance(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check whether a sorted unit agreement scores with the many
    ground truth units it matches are all at chance (it is a false
    positive)

    Args:
        s_id (_type_): sorted unit id
        gt_id (_type_): ground truth unit id
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """

    # make an 1D array
    gt_id = make_1darray(gt_id)

    # start counting scores below chance
    d = 0
    for _, g_i in enumerate(gt_id):
        d += is_score_at_chance(
            scores, s_id, g_i, Sorting_ns, SortingTrue_ns, duration, delta_time
        )
    return d == len(gt_id)


def make_1darray(gt_id):
    if isinstance(gt_id, np.ndarray):
        return gt_id
    else:
        return np.array([gt_id])


def is_oversplitter_1(
    scores, match, s_id, gt_id, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check when sorted unit matches only one ground truth unit

    Args:
        scores (_type_): _description_
        match (_type_): _description_
        s_id (_type_): _description_
        gt_id (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    # find all sorted units that match this ground truth unit
    sorteds = match[match.index == gt_id]["sorted"].values.tolist()

    d = 0
    # loop over sorted unit match
    for _, s_i in enumerate(sorteds):

        # check for other sorted unit match
        # than the target match s_id
        if not s_i == s_id:
            d += is_score_at_chance(
                scores,
                s_i,
                gt_id,
                Sorting_ns,
                SortingTrue_ns,
                duration,
                delta_time,
            )
    return d > 0


def is_oversplitter_2(
    s_id, gt_id, scores, match, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """check when sorted unit matches many ground truth units

    Args:
        s_id (_type_): _description_
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    d = 0
    for _, g_i in enumerate(gt_id):
        d += is_oversplitter_1(
            scores,
            match,
            s_id,
            g_i,
            Sorting_ns,
            SortingTrue_ns,
            duration,
            delta_time,
        )
    return d > 0


def is_poor(det, scores, gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time):
    """is poor (score between chance and threshold for "good" (80%))

    Args:
        scores (_type_): _description_
        gt_id (_type_): _description_
        s_id (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    chance = get_chance_score(
        gt_id, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
    )
    score = get_score(scores, s_id, gt_id)
    return (score > chance) & (score < det)


def is_poor_2(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time, det
):
    """is poor when sorted unit matches many ground truth units
    (all its scores with the ground truth are below the "good" threshold
    and it has at least one score with a ground truth above chance

    Args:
        s_id (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_
        det (_type_): _description_

    Returns:
        _type_: _description_
    """
    chance_all = []
    score_all = []

    for _, g_i in enumerate(gt_id):
        chance_all.append(
            get_chance_score(
                g_i, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
            )
        )
        score_all.append(get_score(scores, s_id, g_i))

    # all scores are below DET
    test_1 = all(np.array(score_all) < det)

    # at least one score is above chance
    test_2 = any(np.array(score_all) > np.array(chance_all))
    return test_1 & test_2


def is_overmerger_2(
    s_id, gt_id, scores, Sorting_ns, SortingTrue_ns, duration, delta_time
):
    """is an overmerger: the sorted unit matches at least two ground
    truths with above chance scores

    Args:
        s_id (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    # get there scores
    chance_all = []
    score_all = []

    for _, g_i in enumerate(gt_id):
        # get chance scores
        chance_all.append(
            get_chance_score(
                g_i, s_id, Sorting_ns, SortingTrue_ns, duration, delta_time
            )
        )
        # get scores
        score_all.append(get_score(scores, s_id, g_i))
    return sum(np.array(score_all) > np.array(chance_all)) > 1


def is_good_2(s_id, gt_id, scores):
    """is sorted unit good: it matches one ground truth with 80% and no
    other ground truth with

    Args:
        s_id (_type_): _description_
        scores (_type_): _description_
        match (_type_): _description_
        Sorting_ns (_type_): _description_
        SortingTrue_ns (_type_): _description_
        duration (_type_): _description_
        delta_time (_type_): _description_

    Returns:
        _type_: _description_
    """
    score_all = []
    for _, g_i in enumerate(gt_id):
        score_all.append(get_score(scores, s_id, g_i))
    return any(np.array(score_all) >= DET)


def set_df(df, sorted, true, quality, score):
    df.loc[sorted, "sorted"] = sorted
    df.loc[sorted, "true"] = true
    df.loc[sorted, "score"] = score
    qual = df.loc[sorted, "quality"]
    # record quality if nan (empty)
    # else append new quality
    if isinstance(qual, str):
        df.loc[sorted, "quality"] += quality
    else:
        df.loc[sorted, "quality"] = quality
    return df


def get_scores(
    SortingTrue,
    Sorting,
    delta_time: float,
):
    comp = comparison.compare_sorter_to_ground_truth(
        SortingTrue,
        Sorting,
        exhaustive_gt=True,
        delta_time=delta_time,
    )
    # return comp.agreement_scores.max(axis=1).sort_values(ascending=False).values
    return comp.agreement_scores

6.985555656397401e-05

In [22]:
scores

,12165,16652,18371,19690,21040,24768,29248,30168,32331,37423,...,4213917,4215563,4216128,4217493,4221920,4223302,4225319,4228700,4229218,4229506
143,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.001927
201,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001315,0.000804,0.000649,0.0,0.001650,0.0,0.0,0.001612,0.0,0.000878
208,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001019,0.002314,0.000000,0.0,0.000391,0.0,0.0,0.000000,0.0,0.000000
215,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.001202,0.000451,0.001184,0.0,0.000970,0.0,0.0,0.000391,0.0,0.000000
217,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000622,0.000282,0.000000,0.0,0.000453,0.0,0.0,0.000599,0.0,0.002375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1612,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000964,0.001316,0.001433,0.0,0.000842,0.0,0.0,0.001188,0.0,0.000590
1613,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.002236,0.000537,0.000000,0.0,0.001676,0.0,0.0,0.000000,0.0,0.000000
1632,0.000109,0.0,0.000109,0.0,0.0,0.0,0.0,0.000109,0.0,0.0,...,0.001555,0.002293,0.001065,0.0,0.001194,0.0,0.0,0.001063,0.0,0.000525


In [24]:
for ix in enumerate()
get_chance_score(4213917, 143, Sorting_ns, SortingTrue_ns, 600, delta_time)

32.481436691116635

In [40]:
SortingTrue_ns.count_num_spikes_per_unit().values()

dict_values([15, 10, 38, 7, 29, 6, 1, 49, 2, 3, 12, 232, 4, 0, 10, 7, 4, 8, 4, 11, 4, 16, 4, 2, 68, 10, 44, 1, 10, 3, 38, 5, 13, 0, 23, 0, 2, 23, 106, 37, 0, 26, 68, 8, 1, 5, 6, 2, 5, 4, 4, 4, 21, 1, 44, 1, 4, 0, 9, 1, 3, 6, 1, 1, 1, 1, 44, 0, 9, 1, 11, 2, 1, 1, 1, 10, 11, 3, 13, 2, 1, 1, 120, 10, 0, 10, 26, 0, 13, 4, 5, 10, 5, 2, 2, 1, 0, 3, 3, 4, 1, 3, 27, 15, 2, 1, 0, 46, 71, 55, 7, 2, 12, 25, 3, 10, 25, 123, 5, 13, 7, 10, 7, 28, 9, 2, 1, 3, 2, 54, 5, 7, 10, 8, 1, 13, 9, 130, 11, 2, 1, 1173, 126, 48, 1, 906, 971, 801, 810, 595, 22, 0, 1, 0, 33, 5, 20, 264, 1, 9, 273, 19, 2, 20, 20, 0, 2, 1, 3, 6, 11, 15, 6, 0, 244, 6, 3, 0, 1, 0, 3, 215, 5, 6, 4, 17, 18, 3, 4, 1, 14, 8, 20, 6, 1090, 2, 320, 27, 2802, 10, 81, 119, 253, 2, 245, 12, 99, 39, 0, 119, 58, 8, 26, 0, 4, 3, 5, 14, 3, 5, 0, 2, 4, 12, 0, 1137, 3, 6, 230, 100, 1, 2, 0, 1, 44, 60, 9, 4, 89, 53, 696, 1, 2, 3, 0, 1, 1, 134, 0, 11, 22, 3, 1, 0, 0, 180, 8, 2, 16, 0, 162, 7, 535, 2, 2, 2, 0, 2195, 866, 3, 0, 1, 159, 8, 4, 0, 2, 6, 17